In [1]:
import urllib.request as urllib2
from bs4 import BeautifulSoup
import pandas as pd
import re
from unidecode import unidecode
import csv
from datetime import datetime

In [2]:
all_pages=[]
link_page = urllib2.urlopen('http://www.metrolyrics.com/taylor-swift-lyrics.html')
soup = BeautifulSoup(link_page, 'html.parser')
pages = soup.find_all('span', attrs={'class': 'pages'})
for x in pages[0].find_all('a'):
    all_pages.append(x['href'])

In [3]:
for link in all_pages:
    scrap_songs(link)

NameError: name 'scrap_songs' is not defined

In [ ]:
def scrap_songs(link):
    
    page= link
    url_page = urllib2.urlopen(page)
    soup = BeautifulSoup(url_page, 'html.parser')
    table_body=soup.find('tbody')
    rows = table_body.find_all('tr')
    names=[]
    for row in rows:    
        matter=row.find_all('a', attrs={'class': 'title hasvidtable'})
        for a in matter:
            Name=a.string
            Name = re.sub(r"\[.*\]\n", "", unidecode(Name))
            Link=a['href']
            
            link_page = urllib2.urlopen(Link)
            soup = BeautifulSoup(link_page, 'html.parser')
            verses = soup.find_all('p', attrs={'class': 'verse'})
            lyrics=''
            for verse in verses:
                text=verse.text.strip()
                text = re.sub(r"\[.*\]\n", "", unidecode(text))
                lyrics=lyrics+' '+text.replace('\n',', ')
        
            print('scrapping'+ Name )
        
            with open('taylor_swift.csv', 'a') as csv_file:
             writer = csv.writer(csv_file)
             writer.writerow([Name, Link, lyrics])

In [ ]:
data=pd.read_csv('ed_sheeran.csv',names = ["Song_Name", "Link", "Lyrics"])

In [5]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop, Adam
from keras.utils.data_utils import get_file
import numpy as np
import pandas as pd
import random
import sys
import io
import re
from unidecode import unidecode

In [6]:
data=pd.read_csv('ed_sheeran.csv',names = ["Song_Name", "Link", "Lyrics"])
data

,Song_Name,Link,Lyrics
0,\nShe Looks So Perfect Lyrics\n,http://www.metrolyrics.com/she-looks-so-perfec...,"Hey, hey, hey, hey, Hey, hey, hey, hey, Hey, ..."
1,\nShape of You Lyrics\n,http://www.metrolyrics.com/shape-of-you-lyrics...,The club isn't the best place to find a lover...
2,\nPerfect Lyrics\n,http://www.metrolyrics.com/perfect-lyrics-ed-s...,"I found a love for me, Darling, just dive rig..."
3,\nPhotograph Lyrics\n,http://www.metrolyrics.com/photograph-lyrics-e...,"Loving can hurt, Loving can hurt sometimes, B..."
4,\nThinking Out Loud Lyrics\n,http://www.metrolyrics.com/thinking-out-loud-l...,When your legs don't work like they used to b...
5,\nHappier Lyrics\n,http://www.metrolyrics.com/happier-lyrics-ed-s...,"Walking down 29th and park, I saw you in anot..."
6,\nA Team Lyrics\n,http://www.metrolyrics.com/a-team-lyrics-ed-sh...,"White lips, pale face, Breathing in snowflake..."
7,\nI See Fire Lyrics\n,http://www.metrolyrics.com/i-see-fire-lyrics-e...,"Oh, misty eye of the mountain below, Keep car..."
8,\nSupermarket Flowers Lyrics\n,http://www.metrolyrics.com/supermarket-flowers...,I took the supermarket flowers from the windo...
9,\nCastle on the Hill Lyrics\n,http://www.metrolyrics.com/castle-on-the-hill-...,"When I was six years old, I broke my leg, I w..."


In [7]:
def get_lines(df):
    corpus=[]
    for index , row in df['Lyrics'].iteritems():
        row=str(row).lower()
        for words in row.split(','):
            new_words = re.findall(r"\b[a-z']+\b", unidecode(words))
            corpus= corpus+ new_words
    return corpus

In [8]:
all_lines=get_lines(data)

In [9]:
SEQ_LENGTH = 50 + 1
sequences = list()

for i in range(SEQ_LENGTH, len(all_lines)):
    seq = all_lines[i - SEQ_LENGTH: i]
    sequences.append(seq)

print('Total Sequences: %d' % len(sequences))

Total Sequences: 62380


In [10]:
corpus= set(all_lines)
word_to_index = {w: i for i, w in enumerate(corpus)}
index_to_word = {i: w for w, i in word_to_index.items()}
word_indices = [word_to_index[word] for word in corpus]
vocab_size = len(corpus)
print('Vocabulary size: {}'.format(len(corpus)))

Vocabulary size: 3043


In [11]:
def get_tokenized_lines(lines, seq_len):
    tokenized = np.zeros((len(lines), seq_len))
    
    for r, line in enumerate(lines):
        for c, word in enumerate(line):
            tokenized[r, c] = word_to_index[word]

    return tokenized

In [12]:
tokenized_seq = get_tokenized_lines(sequences, SEQ_LENGTH)
tokenized_seq

array([[ 591.,  591.,  591., ..., 1084., 1492.,  958.],
       [ 591.,  591.,  591., ..., 1492.,  958., 2820.],
       [ 591.,  591.,  591., ...,  958., 2820.,  482.],
       ...,
       [2128., 2285., 2243., ..., 1812., 2031., 2969.],
       [2285., 2243., 1599., ..., 2031., 2969., 1584.],
       [2243., 1599., 1691., ..., 2969., 1584., 2726.]])

In [13]:
tokenized_seq[:,:-1]

array([[ 591.,  591.,  591., ..., 1020., 1084., 1492.],
       [ 591.,  591.,  591., ..., 1084., 1492.,  958.],
       [ 591.,  591.,  591., ..., 1492.,  958., 2820.],
       ...,
       [2128., 2285., 2243., ...,   14., 1812., 2031.],
       [2285., 2243., 1599., ..., 1812., 2031., 2969.],
       [2243., 1599., 1691., ..., 2031., 2969., 1584.]])

In [14]:
tokenized_seq[:,-1]

array([ 958., 2820.,  482., ..., 2969., 1584., 2726.])

In [15]:
from keras.utils import to_categorical

X, y = tokenized_seq[:, :-1], tokenized_seq[:, -1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = len(X[0])

In [16]:
print("X_shape", X.shape)
print("y_shape", y.shape)

X_shape (62380, 50)
y_shape (62380, 3043)


In [17]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X, y, batch_size=256, epochs=100)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 50)            152150    
_________________________________________________________________
lstm_1 (LSTM)                (None, 50, 100)           60400     
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 3043)              307343    
Total params: 610,393
Trainable params: 610,393
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
62380/62380 [==============================] - 56s 892us/step - loss: 6.1981 - acc: 0.0393
Epoch 2/100
62380/62380 [===

62380/62380 [==============================] - 57s 911us/step - loss: 1.3415 - acc: 0.6917
Epoch 71/100
62380/62380 [==============================] - 57s 909us/step - loss: 1.3171 - acc: 0.6984
Epoch 72/100
62380/62380 [==============================] - 56s 905us/step - loss: 1.3002 - acc: 0.7026
Epoch 73/100
62380/62380 [==============================] - 56s 903us/step - loss: 1.2785 - acc: 0.7064
Epoch 74/100
62380/62380 [==============================] - 56s 903us/step - loss: 1.2571 - acc: 0.7100
Epoch 75/100
62380/62380 [==============================] - 56s 895us/step - loss: 1.2342 - acc: 0.7162
Epoch 76/100
62380/62380 [==============================] - 54s 867us/step - loss: 1.2116 - acc: 0.7218
Epoch 77/100
62380/62380 [==============================] - 54s 869us/step - loss: 1.1916 - acc: 0.7264
Epoch 78/100
62380/62380 [==============================] - 54s 869us/step - loss: 1.1707 - acc: 0.7292
Epoch 79/100
62380/62380 [==============================] - 54s 869us/step - 

In [18]:
model.save('model.h5')

In [19]:
from keras.models import load_model
model=load_model('model.h5')

In [20]:
model.save_weights('weights.h5')
with open('archict.json','w') as f:
    f.write(model.to_json())